In [63]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
from scipy.stats import norm

In [58]:
## Geting data from yfinance (Yahoo OpenSource)

def getData(ticker, rangeTime, plot=None):

    endDate = datetime.now()
    startDate = endDate - relativedelta(months=rangeTime)

    ticker_data = {}

    for t in ticker:

        data = yf.download(t, start=startDate, end=endDate)
        ticker_data[t] = data

        if plot:

            # Plotar os preços de fechamento
            plt.figure(figsize=(10, 6))
            plt.plot(data['Close'], label='Close Price')
            plt.title('Close price for ' + t)
            plt.xlabel('Data')
            plt.ylabel('Price (USD)')
            plt.legend()
            plt.grid(True)
            plt.show()

    return ticker_data

In [ ]:
# Old way to use historical data to calculate Value-At-Risk

def varHistoric(data, alpha):

    '''
    
    Data: Historical data of a asset
    Alpha: The % that will imply to the volatility (Confidence Level = 1 - Alpha)

    '''

    var = {}

    for ticket, ticket_data in data.items():

        ticket_data['returns'] = ticket_data['Close'].pct_change()
        returns = ticket_data['returns'].dropna()
        
        returns_sorted = sorted(returns)

        confidence_index = int((alpha) * len(returns_sorted))

        valueRisk = -returns_sorted[confidence_index]

        var[ticket] = valueRisk

        print(f'The Value at Risk (VaR) with a confidence level of {(1 - alpha) * 100}% is: {var[ticket]:.2%}')

    return var

In [ ]:
# Calculate the Value-At-Risk by the variance

def varVariance(data, alpha):

    '''
    
    Data: Historical data of a asset
    Alpha: The % that will imply to the volatility (Confidence Level = 1 - Alpha)

    '''
    
    var = {}

    for ticket, ticket_data in data.items():

        ticket_data['returns'] = ticket_data['Close'].pct_change()

        returns = ticket_data['returns'].dropna()
        
        mean_return = returns.mean()
        variance = returns.var()

        std_dev = np.sqrt(variance)

        z_score = norm.ppf(alpha)
        valueRisk = -(mean_return + z_score * std_dev)

        var[ticket] = valueRisk

        print(f'The Value at Risk (VaR) with a confidence level of {(1 - alpha) * 100}% is: {var[ticket]:.2%}')

    return var

In [139]:
ticker = ['AAPL', 'MSFT']
rangeTime = 24

data = getData(ticker, rangeTime, plot=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [138]:
alpha = 0.05

var = varHistoric(data, alpha)

The Value at Risk (VaR) with a confidence level of 95.0% is: 3.05%
The Value at Risk (VaR) with a confidence level of 95.0% is: 2.71%


In [136]:
alpha = 0.05

var = varVariance(data, alpha)

The Value at Risk (VaR) with a confidence level of 95.0% is: 2.91%
The Value at Risk (VaR) with a confidence level of 95.0% is: 2.97%
